In [ ]:
import glob
import numpy as np
import pandas as pd
import warnings
import pickle

from biom import load_table, Table
from gemelli.rpca import rpca_table_processing
from tqdm.notebook import tqdm
from skbio import OrdinationResults
from skbio.stats.composition import closure
from scipy.stats import mannwhitneyu

%matplotlib inline
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
#load metadata
metadata = pd.read_csv('../../data/simulations/ihmp/sample-metadata-plus-train-tests-last.csv', index_col=0) #68 samples (last tp)
print(metadata.diagnosis.value_counts())

In [ ]:
tables = {omics_.split('/')[-1].split('.')[0]:rpca_table_processing(load_table(omics_),
                                                                    min_sample_count=0,
                                                                    min_feature_count=0,
                                                                    min_feature_frequency=0)
          for omics_ in glob.glob('../../data/simulations/ihmp/*.biom')
          if not any ('_' + str(d_) in omics_ for d_ in [11, 9, 7, 5, 3])}
tables['meta_g_taxonomic_profiles'] = load_table('../../data/simulations/ihmp/meta_g_taxonomic_profiles.biom')
tables.pop('shared_meta_g_taxonomic_profiles')

# re-close metaG/T data
shared_samps = set.intersection(*[set(t_.ids()) for t_ in tables.values()]) & set(metadata.index)
tables = {t_k:t_.filter(shared_samps) for t_k, t_ in tables.items()}
for t_ in ['meta_g_taxonomic_profiles','meta_t_ecs']:
    tbl_tmp = tables[t_].to_dataframe().copy().apply(closure)
    tables[t_] = Table(tbl_tmp.values, tbl_tmp.index, tbl_tmp.columns)

In [ ]:
#add subsampled table
for d_ in [11, 9, 7, 5, 3]:
   tables['meta_g_taxonomic_profiles_%i' % (d_)] = load_table('../../data/simulations/ihmp/meta_g_taxonomic_profiles_%i_last.biom' % (d_))
#sanity check
tables

In [ ]:
#load modality to feature dict
with open('../../data/simulations/ihmp/mod_to_feature_map.pkl', 'rb') as f:
   mod_features = pickle.load(f)

In [ ]:
joint_weights = {}
joint_best_pc = {}

for k in tqdm(glob.glob('../../results/simulations/ihmp/joint-rpca-results/last_splits/*ord*')):
        
    fold_ = k.split('/')[-1].split('-')[0]
    s_mod_ = k.split('/')[-1].split('-')[1]

    if "meta_g_taxonomic_profiles_" in s_mod_:
        sparsity_ = k.split('/')[-1].split('-')[1]
    else:
        sparsity_ = 'meta_g_taxonomic_profiles'

    #load ordination results
    df_ = OrdinationResults.read(k)

    #feature loadings
    df_weight_ = df_.features
    df_weight_.columns = ['PC1','PC2','PC3']
    joint_weights[(fold_, sparsity_)] = df_weight_

    #subject loadings
    df_ord_ = df_.samples
    df_ord_.columns = ['PC1','PC2','PC3']
    df_ord_ = pd.concat([df_ord_, metadata], axis=1, sort=False)
    
    #select pc that differentiates IBD best
    p_vals_ = {}
    for pc_ in ['PC1','PC2','PC3']:
        f_, p_ = mannwhitneyu(df_ord_[df_ord_.diagnosis == 'IBD'].dropna(subset=[pc_])[pc_].values,
                              df_ord_[df_ord_.diagnosis == 'nonIBD'].dropna(subset=[pc_])[pc_].values)
        p_vals_[pc_] = round(p_, 5)

    best_pc = min(p_vals_, key=p_vals_.get)
    print(fold_, sparsity_)
    print(p_vals_, best_pc)
    #keep track of best pc for specific fold and sparsity
    joint_best_pc[(fold_, sparsity_)] = best_pc
    print()

In [ ]:
mofa_weights = {}
mofa_maps = {}
for k in tqdm(glob.glob('../../results/simulations/ihmp/mofa/last_splits/*weights.model.*')):
    
    fold_ = k.split('/')[-1].split('.')[0]
    s_mod_ = k.split('/')[-1].split('.')[3]

    if s_mod_=='':
        sparsity_ = 'meta_g_taxonomic_profiles'
        feats_map = pd.read_csv(
            "../../results/simulations/ihmp/stacked-tables-last/fold-{}-subset-test-meta_g_taxonomic_profiles-map.tsv.gz".format(fold_), #for last tps
            sep='\t', index_col=0)
        feats_map = feats_map.drop_duplicates(subset='feature').copy()
        feats_map['feature'] = feats_map['feature'].astype(str)
    else:
        sparsity_ = 'meta_g_taxonomic_profiles_{}'.format(s_mod_)
        feats_map = pd.read_csv(
            "../../results/simulations/ihmp/stacked-tables-last-sims/fold-{}-subset-test-{}-map.tsv.gz".format(fold_, sparsity_), 
            sep='\t', index_col=0)
        feats_map = feats_map.drop_duplicates(subset='feature').copy()
        feats_map['feature'] = feats_map['feature'].astype(str)
    
    mofa_maps [(fold_, sparsity_)] = feats_map

    df_ = pd.read_csv(k, index_col=0)
    df_ = pd.pivot_table(columns='factor', values='value', index='feature', data=df_)
    #add to metadata for ML
    df_ = df_[['Factor1','Factor2','Factor3']]
    df_.columns = ['PC1','PC2','PC3']
    mofa_weights[(fold_, sparsity_)] = df_

In [ ]:
mofa_best_pc = {}
for k in tqdm(glob.glob('../../results/simulations/ihmp/mofa/last_splits/*factors.model.*')):
    
    fold_ = k.split('/')[-1].split('.')[0]
    s_mod_ = k.split('/')[-1].split('.')[3]
    if s_mod_=='':
        sparsity_ = 'meta_g_taxonomic_profiles'
    else:
        sparsity_ = 'meta_g_taxonomic_profiles_{}'.format(s_mod_)
    
    df_ = pd.read_csv(k, index_col=0)
    df_ = pd.pivot_table(columns='factor', values='value', index='sample', data=df_)
    #add to metadata for ML
    df_ = df_[['Factor1','Factor2','Factor3']]
    df_.columns = ['PC1','PC2','PC3']
    df_ = pd.concat([df_, metadata], axis=1, sort=False)
    
    #find the PC that drives IBD clustering
    p_vals_ = {}
    for pc_ in ['PC1','PC2','PC3']:
        f_, p_ = mannwhitneyu(df_[df_.diagnosis == 'IBD'].dropna(subset=[pc_])[pc_].values,
                              df_[df_.diagnosis != 'IBD'].dropna(subset=[pc_])[pc_].values)
        p_vals_[pc_] = round(p_,5)
    best_pc = min(p_vals_, key=p_vals_.get)
    
    print(fold_, sparsity_)
    print(p_vals_, best_pc)
    mofa_best_pc[(fold_, sparsity_)] = best_pc
    print()

In [ ]:
#set threshold of features to select
n_ = 5 #top 5 + bottom 5, so n=10 per modality

#get top feats per modality for Joint-RPCA
joint_fold_feats = {}
joint_fold_tbls = {}

for (f_, sparsity_), weights_ in joint_weights.items():

    top_feats = []
    bot_feats = []
    tbl_omics = pd.DataFrame()
    best_pc = joint_best_pc[(f_, sparsity_)]
    #sanity check
    if best_pc != 'PC1':
        print(f_, sparsity_, best_pc)

    for omic_, feats_ in mod_features.items():
        
        #get common features in tables and joint-rpca results
        omic_feats_ = set(feats_) & set(weights_.index)
        
        weights_sub_ = weights_.loc[list(omic_feats_)].copy()
        weights_sub_.sort_values(by=best_pc, ascending=False, inplace=True)
        #get top features
        top_ = weights_sub_.head(n_).index
        top_feats.extend(top_)
        
        #get bottom features
        bot_ = weights_sub_.tail(n_).index
        bot_feats.extend(bot_)

        #if MetaG, use the appropriate sparsity version
        if omic_ == 'meta_g_taxonomic_profiles':
            omic_use_ = sparsity_
        else:
            omic_use_ = omic_
        
        #get tbl with raw values for these features
        all_feats_ = np.concatenate([top_, bot_])
        tbl_ = tables[omic_use_].to_dataframe().loc[all_feats_, ]
        tbl_omics = pd.concat([tbl_omics, tbl_.T], axis=1)
        
    joint_fold_feats[(f_, sparsity_)] = (top_feats, bot_feats) #feature list
    joint_fold_tbls[(f_, sparsity_)] = tbl_omics #table with raw counts

In [ ]:
# save
# with open("../../data/simulations/ihmp/feature-overlap/joint-last-feat-list-50.pkl", "wb") as f:
#     pickle.dump(joint_fold_feats, f)

# with open("../../data/simulations/ihmp/feature-overlap/joint-last-feat-tbls-50.pkl", "wb") as f:
#     pickle.dump(joint_fold_tbls, f)

In [ ]:
#set threshold of features to select
n_ = 5 #top 5 + bottom 5, so n=10 per modality

#get top feats per modality for Joint-RPCA
mofa_fold_feats = {}
mofa_fold_tbls = {}

for (f_, sparsity_), weights_ in mofa_weights.items():
    #create map for the mofa features
    id_map_df = mofa_maps[(f_, sparsity_)]
    id_map_dict = dict(zip(id_map_df['feature'], id_map_df['orig_feature']))
    #get the original feature names
    weights_.index = weights_.index.astype(str)
    weights_['orig_feature'] = weights_.index.map(id_map_dict)
    
    top_feats = []
    bot_feats = []
    tbl_omics = pd.DataFrame()

    best_pc = mofa_best_pc[(f_, sparsity_)]
    #sanity check
    if best_pc != 'PC1':
        print(f_, sparsity_, best_pc)

    for omic_ in id_map_df['view'].unique():

        feats_df = id_map_df[id_map_df.view==omic_].copy()
        feats_df = feats_df.drop_duplicates(subset='feature')            
        
        #get common features in tables and mofa results
        feats_ = feats_df.feature.values
        omic_feats_ = list(set(feats_) & set(weights_.index))
        weights_sub_ = weights_.loc[list(omic_feats_)].copy()
        weights_sub_.sort_values(by=best_pc, ascending=False, inplace=True)

        if omic_ == 'HMP2_proteomics_ecs':
            weights_sub_['orig_feature'] = ["Prot " + id for id in weights_sub_['orig_feature']]
        
        #get top features
        top_ = weights_sub_.head(n_).orig_feature.values
        top_feats.extend(top_)
        #get bottom features
        bot_ = weights_sub_.tail(n_).orig_feature.values
        bot_feats.extend(bot_)
        
        #get tbl with raw values for these features
        if omic_ == 'meta_g_taxonomic_profiles':
            omic_use_ = sparsity_
        else:
            omic_use_ = omic_
        
        #get tbl with raw values for these features
        all_feats_ = np.concatenate([top_, bot_])
        tbl_ = tables[omic_use_].to_dataframe().loc[all_feats_, ]
        tbl_omics = pd.concat([tbl_omics, tbl_.T], axis=1)
        #print(tbl_omics.shape)

    mofa_fold_feats[(f_, sparsity_)] = (top_feats, bot_feats)
    mofa_fold_tbls[(f_, sparsity_)] = tbl_omics

In [ ]:
# save
# with open("../../data/simulations/ihmp/feature-overlap/mofa-last-feat-list-50.pkl", "wb") as f:
#     pickle.dump(mofa_fold_feats, f)

# with open("../../data/simulations/ihmp/feature-overlap/mofa-last-feat-tbl-50.pkl", "wb") as f:
#     pickle.dump(mofa_fold_tbls, f)